# Naive Bayes

In [1]:
import csv
import random
import math

In [2]:
def loadcsv(filename):
    lines = csv.reader(open(filename, "r"));
    dataset = list(lines)
    for i in range(len(dataset)):
        dataset[i] = [float(x) for x in dataset[i]]

    return dataset

def splitdataset(dataset, splitratio):
    trainsize = int(len(dataset) * splitratio);
    trainset = []
    copy = list(dataset)

    while len(trainset) < trainsize:
        index = random.randrange(len(copy));
        trainset.append(copy.pop(index))
        
    return [trainset, copy]

def separatebyclass(dataset):
    separated = {}

    for i in range(len(dataset)):
        vector = dataset[i]
        if (vector[-1] not in separated):
            separated[vector[-1]] = []
        separated[vector[-1]].append(vector)
        
    return separated

In [3]:
def mean(numbers):
    return sum(numbers)/float(len(numbers))

def stdev(numbers):
    avg = mean(numbers)
    variance = sum([pow(x-avg,2) for x in numbers])/float(len(numbers)-1)
    return math.sqrt(variance)

In [4]:
def summarize(dataset): 
    summaries = [(mean(attribute), stdev(attribute)) for attribute in zip(*dataset)];
    del summaries[-1] 
    return summaries

def summarizebyclass(dataset):
    separated = separatebyclass(dataset);

    summaries = {}
    for classvalue, instances in separated.items():
        summaries[classvalue] = summarize(instances)

    return summaries   

In [5]:
def calculateprobability(x, mean, stdev):
    exponent = math.exp(-(math.pow(x-mean,2)/(2*math.pow(stdev,2))))
    return (1 / (math.sqrt(2*math.pi) * stdev)) * exponent

def calculateclassprobabilities(summaries, inputvector):
    probabilities = {}
    for classvalue, classsummaries in summaries.items():
        probabilities[classvalue] = 1
        for i in range(len(classsummaries)):
            mean, stdev = classsummaries[i] 
            x = inputvector[i] 
            probabilities[classvalue] *= calculateprobability(x, mean, stdev)
        return probabilities

In [6]:
def predict(summaries, inputvector): 
    probabilities = calculateclassprobabilities(summaries, inputvector)
    bestLabel, bestProb = None, -1

    for classvalue, probability in probabilities.items():
        if bestLabel is None or probability > bestProb:
            bestProb = probability
            bestLabel = classvalue

    return bestLabel

def getpredictions(summaries, testset):
    predictions = []

    for i in range(len(testset)):
        result = predict(summaries, testset[i])
        predictions.append(result)
        
    return predictions

In [7]:
def getaccuracy(testset, predictions):
    correct = 0
    for i in range(len(testset)):
        if testset[i][-1] == predictions[i]:
            correct += 1

    return (correct/float(len(testset))) * 100.0

## Main

In [8]:
splitratio = 0.7
dataset = loadcsv("naivedata.csv");

trainingset, testset = splitdataset(dataset, splitratio)
print(f'Split {len(dataset)} rows into train = {len(trainingset)} and test = {len(testset)} rows')

summaries = summarizebyclass(trainingset)

predictions = getpredictions(summaries, testset)

accuracy = getaccuracy(testset, predictions)
print(f'Accuracy of the classifier is : {accuracy}%')

Split 768 rows into train = 537 and test = 231 rows
Accuracy of the classifier is : 69.6969696969697%


# Sklearn

In [9]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn import metrics

# Load data
df = pd.read_csv("naivedata.csv")
feature_col_names = df.columns[:-1]
predicted_class_names = df.columns[-1]

X = df[feature_col_names].values
y = df[predicted_class_names].values

# Prepare data
xtrain, xtest, ytrain, ytest = train_test_split(X, y, test_size=0.33)
print ('The total number of Training Data :',ytrain.shape)
print ('The total number of Test Data :',ytest.shape)

# Train
clf = GaussianNB().fit(xtrain,ytrain.ravel())
predicted = clf.predict(xtest)
predictTestData= clf.predict([[6,148,72,35,0,33.6,0.627,50]])

# Analysis
print('\nConfusion matrix')
print(metrics.confusion_matrix(ytest, predicted))
print('\nAccuracy of the classifier is', metrics.accuracy_score(ytest, predicted)*100)
print('The value of Precision:', metrics.precision_score(ytest, predicted))
print('The value of Recall:', metrics.recall_score(ytest, predicted))
print("Predicted Value for individual Test Data:", predictTestData)

The total number of Training Data : (513,)
The total number of Test Data : (254,)

Confusion matrix
[[151  31]
 [ 27  45]]

Accuracy of the classifier is 77.16535433070865
The value of Precision: 0.5921052631578947
The value of Recall: 0.625
Predicted Value for individual Test Data: [1]
